## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. Please submit only the `*.ipynb` file.

4. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

5. Fill your group name and collaborators below:

In [1]:
GROUPNAME = "35"
COLLABORATORS = "Yevhen Yazvinskyi 396417, Sourabh Raj 397371, Yizhou Peng 398821"

---

# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree. Should return `"less"` or `"more"`.
* Test your function on the tuple `('yes', 31, 'good')`,

In [2]:
def decision(x):
    # >>>>> YOUR CODE HERE
    if (x[0]=='yes'):
        if(x[1]>29.5):
            return 'more'
        return 'less'
    if (x[2]=='good'):
        return 'less'
    return 'more'
    # <<<<< END YOUR CODE

In [3]:
# Test
x = ('yes', 31, 'good')
assert decision(x) == 'more'


## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [4]:
def gettest():
    # >>>>> YOUR CODE HERE
    result = list()
    with open('health-test.txt', 'r') as f:
        for line in f:
            a,b,c = line[:-1].split(',')
            result.append((a,float(b),c))
    return result
    # <<<<< END YOUR CODE

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and return the percentage of them that are classified as "more".

In [5]:
def evaluate_testset():
    # >>>>> YOUR CODE HERE
    counted = 0
    summ = 0
    for x in gettest():
        summ += 1
        if decision(x) =='more': counted += 1
    return float(counted/summ)
    # <<<<< END YOUR CODE

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [6]:
def gettrain():
    # >>>>> YOUR CODE HERE
    result = list()
    with open('health-train.txt', 'r') as f:
        for line in f:
            a,b,c,d = line[:-1].split(',')
            result.append(((a,float(b),c),d))
    return result
    # <<<<< END YOUR CODE

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a, b) = (a[0] != b[0]) + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] != b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes', 31, 'good')`

In [7]:
def neighbor(x, trainset):
    # >>>>> YOUR CODE HERE
    point = trainset[0]
    point_params = point[0]
    distance = (x[0] != point_params[0]) + ((x[1] - point_params[1]) / 50.0) ** 2 + (x[2] != point_params[2])
    neighbor_distance = distance
    neighbor = point
    for point in trainset[1:]:
        point_params = point[0]
        distance = (x[0] != point_params[0]) + ((x[1] - point_params[1]) / 50.0) ** 2 + (x[2] != point_params[2])
        if (distance < neighbor_distance):
            neighbor_distance = distance
            neighbor = point
    return neighbor[1]
    # <<<<< END YOUR CODE

In [8]:
# Test
x = ('yes', 31, 'good')
assert neighbor(x, gettrain()) == "more"


* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and with which probability it happens.

In [9]:
def compare():
    # >>>>> YOUR CODE HERE
    Xdisagree = list()
    trained_set = gettrain()
    for elem in gettest():
        if (neighbor(elem,trained_set)!=decision(elem)):
            Xdisagree.append(elem)
    probability = len(Xdisagree) / len(gettest())
    # <<<<< END YOUR CODE
    return Xdisagree, probability

In [10]:
Xdisagree, probability = compare()
assert type(Xdisagree) == list
assert probability >= 0.0 and probability <= 1.0

One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [11]:
class NearestMeanClassifier:
    def train(self, dataset):
        # >>>>> YOUR CODE HERE
        self.more_avg = [0,0,0]
        self.less_avg = [0,0,0]
        trained_set = dataset
        trained_more = 0
        trained_less = 0
        for elem in trained_set:
            if elem[1] == 'more':
                self.more_avg[0] += elem[0][0] == 'yes'
                self.more_avg[1] += elem[0][1]
                self.more_avg[2] += elem[0][2] == 'poor'
                trained_more +=1
            else:
                self.less_avg[0] += elem[0][0] == 'yes'
                self.less_avg[1] += elem[0][1]
                self.less_avg[2] += elem[0][2] == 'poor'
                trained_less +=1
        self.more_avg = (self.more_avg[0]/trained_more,self.more_avg[1]/trained_more,self.more_avg[2]/trained_more)
        self.less_avg = (self.less_avg[0]/trained_less,self.less_avg[1]/trained_less,self.less_avg[2]/trained_less)
        # <<<<< END YOUR CODE

    def predict(self, x):
        # >>>>> YOUR CODE HERE
        dist_more = ((x[0] == 'yes') - self.more_avg[0]) ** 2 + ((x[1] - self.more_avg[1]) / 50.0) ** 2 + ((x[2] == 'poor') - self.more_avg[2]) ** 2
        dist_less = ((x[0] == 'yes') - self.less_avg[0]) ** 2 + ((x[1] - self.less_avg[1]) / 50.0) ** 2 + ((x[2] == 'poor') - self.less_avg[2]) ** 2
        prediction = 'less'
        if dist_more < dist_less:
            prediction = 'more'
        # <<<<< END YOUR CODE
        return prediction

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [12]:
def build_and_train():
    # >>>>> YOUR CODE HERE
    classifier = NearestMeanClassifier()
    classifier.train(gettrain())
    print(classifier.more_avg)
    print(classifier.less_avg)
    # <<<<< END YOUR CODE
build_and_train()

(0.5714285714285714, 37.142857142857146, 0.5714285714285714)
(0.3333333333333333, 32.111111111111114, 0.2222222222222222)


* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [13]:
def predict_test():
    # >>>>> YOUR CODE HERE
    agreed_samples = list()
    trained_set = gettrain()
    classifier = NearestMeanClassifier()
    classifier.train(gettrain())
    for elem in gettest():
        if (decision(elem) == neighbor(elem,trained_set) == classifier.predict(elem)):
            agreed_samples.append(elem)
    print (agreed_samples)
    # <<<<< END YOUR CODE
    return agreed_samples
predict_test()

[('no', 50.0, 'good'), ('no', 23.0, 'good'), ('yes', 45.0, 'poor'), ('no', 60.0, 'good'), ('no', 15.0, 'poor'), ('no', 18.0, 'good')]


[('no', 50.0, 'good'),
 ('no', 23.0, 'good'),
 ('yes', 45.0, 'poor'),
 ('no', 60.0, 'good'),
 ('no', 15.0, 'poor'),
 ('no', 18.0, 'good')]